# Imports

In [1]:
import os
import sys
import math
import time
import argparse
import numpy as np

from PIL import Image

from datetime import datetime

import dataGenerator
import logly

import torch
import torch.utils
import torch.utils.data
import torch.nn.parallel
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import lr_scheduler

import torchvision
import torchvision.utils as vutils
from torchvision.transforms import ToPILImage
import torchvision.transforms as transforms
from torchvision import datasets, models, transforms

from livelossplot import PlotLosses

# User Settings

In [2]:
import pickle

# class Company(object):
#     def __init__(self, name, value):
#         self.name = name
#         self.value = value

# with open('company_data.pkl', 'wb') as output:
#     company1 = Company('banana', 40)
#     pickle.dump(company1, output, pickle.HIGHEST_PROTOCOL)

#     company2 = Company('spam', 42)
#     pickle.dump(company2, output, pickle.HIGHEST_PROTOCOL)

# del company1
# del company2

# with open('company_data.pkl', 'rb') as input:
#     company1 = pickle.load(input)
#     print(company1.name)  # -> banana
#     print(company1.value)  # -> 40

#     company2 = pickle.load(input)
#     print(company2.name) # -> spam
#     print(company2.value)  # -> 42

    
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

# sample usage
#save_object(company1, 'company1.pkl')

# Define DataLoader

In [2]:
#Set Values
show_live_loss = False
newTraining = False
save_masks = False
print_loss = True
output_tensors = False
train_flag = True
EPOCHS = 1
iteration = "-1"

# default values
NUM_CLASSES = 1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batchSize = 4
num_batch = 25000 #11112
imageSize = 416
SAVE_LOCATION = "./data/models/model_test"
LOAD_LOCATION = "./data/models/model_best_test"
INTERRUPTED_LOCATION =  "./data/models/model_interrupted"
train_dir = "./data/train/"
val_dir = "./data/validation/"
log_dir = './data/logs/'

from torchvision import datasets

class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """
    # override the __getitem__ method. this is the method dataloader calls
    def __getitem__(self, index):
        
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        
        # the image file path
        path = self.imgs[index][0]
        
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        
        return tuple_with_path

# Load Data

In [3]:
data_transforms = transforms.Compose([transforms.Resize([imageSize,imageSize]),
                                      transforms.ToTensor()
                                     ])

# instantiate the dataset and dataloader
train_dataset = ImageFolderWithPaths(train_dir, transform=data_transforms) # our custom dataset
#val_dataset = ImageFolderWithPaths(val_dir, transform=data_transforms) # our custom dataset

#loads only photos
train_dataloaders = torch.utils.data.DataLoader(train_dataset, batch_size = batchSize, shuffle=True)
#val_dataloaders = torch.utils.data.DataLoader(val_dataset, batch_size = batchSize, shuffle=True)

data_factory = dataGenerator.dataGenerator(IMAGE_SIZE=imageSize)

# Training W/O Val Steps

In [4]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=4):
    since = time.time()
    best_model = None
    best_loss = math.inf
    #model.train()  
    liveloss = PlotLosses()
    model = model.to(device)
    sigmoid_function = nn.Sigmoid()
    batch_log = logly.logly(True, num_epochs, num_batch)
    
    for epoch in range(num_epochs):
        if print_loss:
            print('Epoch {}/{}'.format(epoch + 1, num_epochs), flush=True)
        for batch in range(num_batch):
            epoch_loss = 0
            running_loss = 0.0
            running_corrects = 0
            logs = {}
            
            if print_loss:
                #print('Epoch {}/{}'.format(epoch + 1, num_epochs), flush=True)
                print('Batch {}/{}'.format(batch + 1, num_batch), flush=True)
            
            #BATCH TUPLE
            inputs, labels, paths = next(iter(train_dataloaders))
            inputs = inputs.to(device)
            #labels.to(device)

            # Moved outside paths forloop as labels would be reset
            i = 0
            labels = torch.zeros(batchSize,NUM_CLASSES,imageSize,imageSize).to(device)

            #build ground-truth batch tensor
            for locations in paths:
                #dtype=torch.int64
                ### Why are we using float?
                #labels = torch.zeros(batchSize,NUM_CLASSES,imageSize,imageSize, dtype = torch.float32).to(device)
                loc = locations.replace(".png", ".pt").replace("photos", "tensors")
                labels[i] = torch.load(loc)
                #print("loading tensor:", i,  loc, flush=True)
                #print("tensor max:", torch.max(labels[i]))
                i += 1

            if train_flag:
                phase = 'train'
                #print("Setting model in training mode")
                # Set model to training mode
                model.train() 
            else:
                phase = 'val'
                #print("Setting model in validation mode")
                # Set model to evaluate mode
                model.eval()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
                #build input-truth batch tensor
                outputs = model(inputs)
                #outputs = model(inputs.to(device)).to(device)
                #outputs = model(inputs)
                loss = criterion(outputs, labels) #ground truth comparison

                if phase == 'train':
                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # backward + optimize 
                    loss.backward() #modifies model
                    optimizer.step() 

                # statistics
                epoch_loss = loss.item()

            outputs = sigmoid_function(outputs)

    #         _, preds = torch.max(outputs, 1)
    #         running_loss += loss.detach() * inputs.size(0)
    #         running_corrects += torch.sum(preds == labels.data, dtype = torch.float32)
    #         labels = labels.to(dtype=torch.long)
    #         running_corrects += torch.sum(preds == labels.data)

    #         e_loss = running_loss / len(dataloaders.dataset)
    #         epoch_acc = running_corrects.double() / len(dataloaders.dataset)

    #         print("Running corrects: {}".format(running_corrects), flush=True)


            #logs['accuracy'] = epoch_acc

            batch_log.log(phase, epoch, batch, epoch_loss)
            
            if batch % 20 == 0:
                log_name = str(datetime.now().date()) + iteration
                batch_log.write(log_dir, log_name)

            if show_live_loss and phase == 'train':
                logs['log loss'] = epoch_loss
                logs['accuracy'] = 1 - epoch_loss
                liveloss.update(logs)
                liveloss.draw()

            if save_masks and phase == 'val':
                # need to fix before running
                outputs_dir = "data/results/" + str(datetime.now().date())
                os.makedirs(outputs_dir, exist_ok=True)
                threshold = 0.05
                j = 0
                for locations in paths:
                    img = Image.open(locations)
                    num = locations.replace("/data/photos/", "").replace(".png", "")
                    for threshold in [0.05, .25, .5, .75]:
                        img = data_factory.showMaskOnImage(img,outputs[j], threshold)
                        img.save(outputs_dir  + "/" + str(num) + "-" + str(epoch) + "-" + str(threshold) + ".png","PNG")
                    j += 1

            if print_loss:
                print("loss: {}".format(epoch_loss), flush=True)
                print('---------------', flush=True)
                #input("please enter any key to continue")

            if output_tensors:
                return inputs, outputs, paths

            #save best copy of  model
            #if phase == 'val' and epoch_loss < best_loss:
            if phase == 'train' and epoch_loss < best_loss:
                best_loss = epoch_loss
                torch.save(model, LOAD_LOCATION + iteration)
                torch.save(model.state_dict(), LOAD_LOCATION + "-" + str(datetime.now().date()) + iteration) 
                #torch.save(model, SAVE_LOCATION)
                #torch.save(model, SAVE_LOCATION.replace("model", "model_best"))
        
    time_elapsed = time.time() - since
    
    print('Training complete in {:.1f}m {:.1f}s'.format(time_elapsed // 60, time_elapsed % 60), flush=True)
    
    log_name = str(datetime.now().date()) + iteration
    batch_log.write(log_dir, log_name)
    
    #logs_dir = "data/logs/" + str(datetime.now().date())
    #save_object(log, logs_dir)
    
    #completed model
    torch.save(model,SAVE_LOCATION)
    return model

# Load Pretrained Model Weights

In [5]:
from unet_models import *

#imports related to UNet
if newTraining:
    model = UNet16(num_classes=1, num_filters=32, pretrained=True, is_deconv=True)
    
    print('initializing model with random weights', flush=True)
    torch.nn.init.xavier_uniform_(next(model.center.children())[1].weight)
    
    torch.nn.init.xavier_uniform_(next(model.dec5.children())[1].weight)
    
    torch.nn.init.xavier_uniform_(next(model.dec4.children())[1].weight)

    torch.nn.init.xavier_uniform_(next(model.dec3.children())[1].weight)

    torch.nn.init.xavier_uniform_(next(model.dec2.children())[1].weight)

    torch.nn.init.xavier_uniform_(next(model.dec1.children()).weight)

    torch.nn.init.xavier_uniform_(model.final.weight)
                           
else:
    print("loading weights from", LOAD_LOCATION, flush=True)
    #model = torch.load(SAVE_LOCATION)
    model = torch.load(LOAD_LOCATION)
    #model.load_state_dict(best_model_wts)

criterion = torch.nn.BCEWithLogitsLoss()
#criterion = DICELossMultiClass()
#criterion = IOU_BCELoss()

#Observe adjustments in learning rate
optimizer_ft = optim.Adam(model.parameters(), lr=0.05, weight_decay=0, amsgrad=False, eps=0.1)

# Osscilate between high and low learning rates over time
exp_lr_scheduler = lr_scheduler.CosineAnnealingLR(optimizer_ft, T_max=EPOCHS,eta_min=0.001)  


#model = model.to(device)

loading weights from ./data/models/model_best_test


# Training and Results

In [6]:
try:
    model = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=EPOCHS)
    
except KeyboardInterrupt:
    torch.save(model, INTERRUPTED_LOCATION)
    print('Saved interrupt', flush=True)

Epoch 1/1
Batch 1/25000
loss: 0.3503231704235077
---------------
Batch 2/25000
loss: 0.23570434749126434
---------------
Batch 3/25000
loss: 0.15607358515262604
---------------
Batch 4/25000
loss: 0.15657088160514832
---------------
Batch 5/25000
loss: 0.186697319149971
---------------
Batch 6/25000
loss: 0.1498991698026657
---------------
Batch 7/25000
loss: 0.29338935017585754
---------------
Batch 8/25000
loss: 0.22370532155036926
---------------
Batch 9/25000
loss: 0.17712940275669098
---------------
Batch 10/25000
loss: 0.16079752147197723
---------------
Batch 11/25000
loss: 0.19783130288124084
---------------
Batch 12/25000
loss: 0.2714320421218872
---------------
Batch 13/25000
loss: 0.11483961343765259
---------------
Batch 14/25000
loss: 0.20052070915699005
---------------
Batch 15/25000
loss: 0.17496858537197113
---------------
Batch 16/25000
loss: 0.18933068215847015
---------------
Batch 17/25000
loss: 0.2702680230140686
---------------
Batch 18/25000
loss: 0.1449234783649

Batch 145/25000
loss: 0.17546771466732025
---------------
Batch 146/25000
loss: 0.1636548489332199
---------------
Batch 147/25000
loss: 0.14619146287441254
---------------
Batch 148/25000
loss: 0.1618967056274414
---------------
Batch 149/25000
loss: 0.10615057498216629
---------------
Batch 150/25000
loss: 0.17871326208114624
---------------
Batch 151/25000
loss: 0.2644842863082886
---------------
Batch 152/25000
loss: 0.15729877352714539
---------------
Batch 153/25000
loss: 0.14949411153793335
---------------
Batch 154/25000
loss: 0.17349809408187866
---------------
Batch 155/25000
loss: 0.11137708276510239
---------------
Batch 156/25000
loss: 0.09187893569469452
---------------
Batch 157/25000
loss: 0.1897733360528946
---------------
Batch 158/25000
loss: 0.17483149468898773
---------------
Batch 159/25000
loss: 0.09816595166921616
---------------
Batch 160/25000
loss: 0.18157729506492615
---------------
Batch 161/25000
loss: 0.1709129959344864
---------------
Batch 162/25000
los

Batch 287/25000
loss: 0.1146141067147255
---------------
Batch 288/25000
loss: 0.15647993981838226
---------------
Batch 289/25000
loss: 0.12058486044406891
---------------
Batch 290/25000
loss: 0.21045762300491333
---------------
Batch 291/25000
loss: 0.1603955775499344
---------------
Batch 292/25000
loss: 0.20166291296482086
---------------
Batch 293/25000
loss: 0.1695881187915802
---------------
Batch 294/25000
loss: 0.15557344257831573
---------------
Batch 295/25000
loss: 0.15194477140903473
---------------
Batch 296/25000
loss: 0.21730294823646545
---------------
Batch 297/25000
loss: 0.14763061702251434
---------------
Batch 298/25000
loss: 0.24185770750045776
---------------
Batch 299/25000
loss: 0.12571080029010773
---------------
Batch 300/25000
loss: 0.11062035709619522
---------------
Batch 301/25000
loss: 0.43900611996650696
---------------
Batch 302/25000
loss: 0.10861605405807495
---------------
Batch 303/25000
loss: 0.19925373792648315
---------------
Batch 304/25000
l

Batch 429/25000
loss: 0.09986669570207596
---------------
Batch 430/25000
loss: 0.16060557961463928
---------------
Batch 431/25000
loss: 0.1833087056875229
---------------
Batch 432/25000
loss: 0.13396136462688446
---------------
Batch 433/25000
loss: 0.13536903262138367
---------------
Batch 434/25000
loss: 0.20953157544136047
---------------
Batch 435/25000
loss: 0.11996426433324814
---------------
Batch 436/25000
loss: 0.14783436059951782
---------------
Batch 437/25000
loss: 0.15424010157585144
---------------
Batch 438/25000
loss: 0.13936841487884521
---------------
Batch 439/25000
loss: 0.18007943034172058
---------------
Batch 440/25000
loss: 0.13252602517604828
---------------
Batch 441/25000
loss: 0.1288568675518036
---------------
Batch 442/25000
loss: 0.15633361041545868
---------------
Batch 443/25000
loss: 0.14155414700508118
---------------
Batch 444/25000
loss: 0.1833348423242569
---------------
Batch 445/25000
loss: 0.19116753339767456
---------------
Batch 446/25000
l

loss: 0.11536485701799393
---------------
Batch 572/25000
loss: 0.1920097917318344
---------------
Batch 573/25000
loss: 0.29433485865592957
---------------
Batch 574/25000
loss: 0.21127283573150635
---------------
Batch 575/25000
loss: 0.23678728938102722
---------------
Batch 576/25000
loss: 0.16058577597141266
---------------
Batch 577/25000
loss: 0.1404116004705429
---------------
Batch 578/25000
loss: 0.19892922043800354
---------------
Batch 579/25000
loss: 0.19835135340690613
---------------
Batch 580/25000
loss: 0.2117273062467575
---------------
Batch 581/25000
loss: 0.16258931159973145
---------------
Batch 582/25000
loss: 0.37158143520355225
---------------
Batch 583/25000
loss: 0.23089198768138885
---------------
Batch 584/25000
loss: 0.1605660319328308
---------------
Batch 585/25000
loss: 0.3624477684497833
---------------
Batch 586/25000
loss: 0.1743014007806778
---------------
Batch 587/25000
loss: 0.19720615446567535
---------------
Batch 588/25000
loss: 0.185783982276

...

---------------
Batch 24418/25000
loss: 0.6934197545051575
---------------
Batch 24419/25000
loss: 0.6939515471458435
---------------
Batch 24420/25000
loss: 0.692590057849884
---------------
Batch 24421/25000
loss: 0.6984736323356628
---------------
Batch 24422/25000
loss: 0.6941674947738647
---------------
Batch 24423/25000
loss: 0.6947420835494995
---------------
Batch 24424/25000
loss: 0.693926990032196
---------------
Batch 24425/25000
loss: 0.6917191743850708
---------------
Batch 24426/25000
loss: 0.6947134137153625
---------------
Batch 24427/25000
loss: 0.6940586566925049
---------------
Batch 24428/25000
loss: 0.6936661601066589
---------------
Batch 24429/25000
loss: 0.6935362815856934
---------------
Batch 24430/25000
loss: 0.6932535171508789
---------------
Batch 24431/25000
loss: 0.6928144693374634
---------------
Batch 24432/25000
loss: 0.6930082440376282
---------------
Batch 24433/25000
loss: 0.6932267546653748
---------------
Batch 24434/25000
loss: 0.6931604743003845

Batch 24557/25000
loss: 0.6947417259216309
---------------
Batch 24558/25000
loss: 0.6978007555007935
---------------
Batch 24559/25000
loss: 0.6945468187332153
---------------
Batch 24560/25000
loss: 0.6986408829689026
---------------
Batch 24561/25000
loss: 0.6909487247467041
---------------
Batch 24562/25000
loss: 0.6978440880775452
---------------
Batch 24563/25000
loss: 0.696667492389679
---------------
Batch 24564/25000
loss: 0.690441906452179
---------------
Batch 24565/25000
loss: 0.6901572942733765
---------------
Batch 24566/25000
loss: 0.6930458545684814
---------------
Batch 24567/25000
loss: 0.6935054659843445
---------------
Batch 24568/25000
loss: 0.6904817223548889
---------------
Batch 24569/25000
loss: 0.6929008364677429
---------------
Batch 24570/25000
loss: 0.6934146285057068
---------------
Batch 24571/25000
loss: 0.6913692951202393
---------------
Batch 24572/25000
loss: 0.691581130027771
---------------
Batch 24573/25000
loss: 0.6907234787940979
---------------


loss: 0.6915525794029236
---------------
Batch 24697/25000
loss: 0.6880112886428833
---------------
Batch 24698/25000
loss: 0.696640133857727
---------------
Batch 24699/25000
loss: 0.6973461508750916
---------------
Batch 24700/25000
loss: 0.6969918608665466
---------------
Batch 24701/25000
loss: 0.6895689964294434
---------------
Batch 24702/25000
loss: 0.6941917538642883
---------------
Batch 24703/25000
loss: 0.6937475800514221
---------------
Batch 24704/25000
loss: 0.6906304955482483
---------------
Batch 24705/25000
loss: 0.691281795501709
---------------
Batch 24706/25000
loss: 0.6911706328392029
---------------
Batch 24707/25000
loss: 0.6940311789512634
---------------
Batch 24708/25000
loss: 0.6967048048973083
---------------
Batch 24709/25000
loss: 0.6948934197425842
---------------
Batch 24710/25000
loss: 0.6899847388267517
---------------
Batch 24711/25000
loss: 0.6936116814613342
---------------
Batch 24712/25000
loss: 0.6964278817176819
---------------
Batch 24713/25000

---------------
Batch 24836/25000
loss: 0.6934912800788879
---------------
Batch 24837/25000
loss: 0.6911306977272034
---------------
Batch 24838/25000
loss: 0.6909332275390625
---------------
Batch 24839/25000
loss: 0.6947489976882935
---------------
Batch 24840/25000
loss: 0.6922947764396667
---------------
Batch 24841/25000
loss: 0.6949443817138672
---------------
Batch 24842/25000
loss: 0.6926103234291077
---------------
Batch 24843/25000
loss: 0.6938589811325073
---------------
Batch 24844/25000
loss: 0.691196858882904
---------------
Batch 24845/25000
loss: 0.6951559782028198
---------------
Batch 24846/25000
loss: 0.6936749219894409
---------------
Batch 24847/25000
loss: 0.6899803876876831
---------------
Batch 24848/25000
loss: 0.6937053799629211
---------------
Batch 24849/25000
loss: 0.6948571801185608
---------------
Batch 24850/25000
loss: 0.6938369274139404
---------------
Batch 24851/25000
loss: 0.6932053565979004
---------------
Batch 24852/25000
loss: 0.693177163600921

Batch 24975/25000
loss: 0.6943183541297913
---------------
Batch 24976/25000
loss: 0.7018100023269653
---------------
Batch 24977/25000
loss: 0.6950457692146301
---------------
Batch 24978/25000
loss: 0.6890611052513123
---------------
Batch 24979/25000
loss: 0.6918236613273621
---------------
Batch 24980/25000
loss: 0.6873360276222229
---------------
Batch 24981/25000
loss: 0.6921325325965881
---------------
Batch 24982/25000
loss: 0.6953132152557373
---------------
Batch 24983/25000
loss: 0.6906093955039978
---------------
Batch 24984/25000
loss: 0.6878046989440918
---------------
Batch 24985/25000
loss: 0.6935246586799622
---------------
Batch 24986/25000
loss: 0.6916568279266357
---------------
Batch 24987/25000
loss: 0.6876351833343506
---------------
Batch 24988/25000
loss: 0.6955046653747559
---------------
Batch 24989/25000
loss: 0.6920725703239441
---------------
Batch 24990/25000
loss: 0.6965922117233276
---------------
Batch 24991/25000
loss: 0.696040153503418
--------------

# Testing Outputs

In [10]:
try:
    inputs, outputs, paths = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=EPOCHS)
    #batch = 
except KeyboardInterrupt:
    torch.save(model, INTERRUPTED_LOCATION)
    print('Saved interrupt', flush=True)

# for tensor in labels:
#     #tensor = labels[0]
#     tensor_max_value = torch.max(tensor)
#     print(tensor_max_value)
#     print(type(tensor_max_value.item()))
    

In [11]:
dat = dataGenerator.dataGenerator(416)
j = 0
outputs_dir = "data/outputs/" + str(datetime.now().date())
os.makedirs(outputs_dir, exist_ok=True)
threshold = 0.05
batch = 3

for locations in paths:
    img = Image.open(locations)
    img = dat.showMaskOnImage(img,outputs[j], threshold)
    img.save(outputs_dir  + "/batch_" + str(batch) + "-" + str(j) + ".png","PNG")
    #img.show()
    j += 1

In [ ]:
outputs[0]

In [12]:
locations = "/data/photos/999.png"
loc = locations.replace("/data/photos/", "").replace(".png", "")

print(locations)
print(loc)

/data/photos/999.png
999


In [2]:
import sys
print(sys.version)

3.6.5 |Anaconda, Inc.| (default, Apr 29 2018, 16:14:56) 
[GCC 7.2.0]
